In [ ]:
"""Fairly basic set of tools for real-time data augmentation on image data.
Can easily be extended to include new transformations,
new preprocessing methods, etc...
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import multiprocessing.pool
import os
import re
import threading
import warnings
from functools import partial

import numpy as np
import scipy.ndimage as ndi
from keras import backend as K
from keras.utils.data_utils import Sequence
from scipy import linalg
from six.moves import range

try:
    from PIL import ImageEnhance
    from PIL import Image as pil_image
except ImportError:
    pil_image = None

if pil_image is not None:
    _PIL_INTERPOLATION_METHODS = {
        'nearest': pil_image.NEAREST,
        'bilinear': pil_image.BILINEAR,
        'bicubic': pil_image.BICUBIC,
    }
    # These methods were only introduced in version 3.4.0 (2016).
    if hasattr(pil_image, 'HAMMING'):
        _PIL_INTERPOLATION_METHODS['hamming'] = pil_image.HAMMING
    if hasattr(pil_image, 'BOX'):
        _PIL_INTERPOLATION_METHODS['box'] = pil_image.BOX
    # This method is new in version 1.1.3 (2013).
    if hasattr(pil_image, 'LANCZOS'):
        _PIL_INTERPOLATION_METHODS['lanczos'] = pil_image.LANCZOS


def random_rotation(x, rg, row_axis=1, col_axis=2, channel_axis=0,
                    fill_mode='nearest', cval=0.):
    """Performs a random rotation of a Numpy image tensor.
    # Arguments
        x: Input tensor. Must be 3D.
        rg: Rotation range, in degrees.
        row_axis: Index of axis for rows in the input tensor.
        col_axis: Index of axis for columns in the input tensor.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
    # Returns
        Rotated Numpy image tensor.
    """
    theta = np.deg2rad(np.random.uniform(-rg, rg))
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])

    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x


def random_shift(x, wrg, hrg, row_axis=1, col_axis=2, channel_axis=0,
                 fill_mode='nearest', cval=0.):
    """Performs a random spatial shift of a Numpy image tensor.
    # Arguments
        x: Input tensor. Must be 3D.
        wrg: Width shift range, as a float fraction of the width.
        hrg: Height shift range, as a float fraction of the height.
        row_axis: Index of axis for rows in the input tensor.
        col_axis: Index of axis for columns in the input tensor.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
    # Returns
        Shifted Numpy image tensor.
    """
    h, w = x.shape[row_axis], x.shape[col_axis]
    tx = np.random.uniform(-hrg, hrg) * h
    ty = np.random.uniform(-wrg, wrg) * w
    translation_matrix = np.array([[1, 0, tx],
                                   [0, 1, ty],
                                   [0, 0, 1]])

    transform_matrix = translation_matrix  # no need to do offset
    x = apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x


def random_shear(x, intensity, row_axis=1, col_axis=2, channel_axis=0,
                 fill_mode='nearest', cval=0.):
    """Performs a random spatial shear of a Numpy image tensor.
    # Arguments
        x: Input tensor. Must be 3D.
        intensity: Transformation intensity in degrees.
        row_axis: Index of axis for rows in the input tensor.
        col_axis: Index of axis for columns in the input tensor.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
    # Returns
        Sheared Numpy image tensor.
    """
    shear = np.deg2rad(np.random.uniform(-intensity, intensity))
    shear_matrix = np.array([[1, -np.sin(shear), 0],
                             [0, np.cos(shear), 0],
                             [0, 0, 1]])

    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = transform_matrix_offset_center(shear_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x


def random_zoom(x, zoom_range, row_axis=1, col_axis=2, channel_axis=0,
                fill_mode='nearest', cval=0.):
    """Performs a random spatial zoom of a Numpy image tensor.
    # Arguments
        x: Input tensor. Must be 3D.
        zoom_range: Tuple of floats; zoom range for width and height.
        row_axis: Index of axis for rows in the input tensor.
        col_axis: Index of axis for columns in the input tensor.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
    # Returns
        Zoomed Numpy image tensor.
    # Raises
        ValueError: if `zoom_range` isn't a tuple.
    """
    if len(zoom_range) != 2:
        raise ValueError('`zoom_range` should be a tuple or list of two floats. '
                         'Received arg: ', zoom_range)

    if zoom_range[0] == 1 and zoom_range[1] == 1:
        zx, zy = 1, 1
    else:
        zx, zy = np.random.uniform(zoom_range[0], zoom_range[1], 2)
    zoom_matrix = np.array([[zx, 0, 0],
                            [0, zy, 0],
                            [0, 0, 1]])

    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = transform_matrix_offset_center(zoom_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x


def random_channel_shift(x, intensity, channel_axis=0):
    x = np.rollaxis(x, channel_axis, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_channel + np.random.uniform(-intensity, intensity), min_x, max_x)
                      for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x


def random_brightness(x, brightness_range):
    if len(brightness_range) != 2:
        raise ValueError('`brightness_range should be tuple or list of two floats. '
                         'Received arg: ', brightness_range)

    x = array_to_img(x)
    x = imgenhancer_Brightness = ImageEnhance.Brightness(x)
    u = np.random.uniform(brightness_range[0], brightness_range[1])
    x = imgenhancer_Brightness.enhance(u)
    x = img_to_array(x)
    return x


def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix


def apply_transform(x,
                    transform_matrix,
                    channel_axis=0,
                    fill_mode='nearest',
                    cval=0.):
    """Apply the image transformation specified by a matrix.
    # Arguments
        x: 2D numpy array, single image.
        transform_matrix: Numpy array specifying the geometric transformation.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
    # Returns
        The transformed version of the input.
    """
    x = np.rollaxis(x, channel_axis, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(
        x_channel,
        final_affine_matrix,
        final_offset,
        order=1,
        mode=fill_mode,
        cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x


def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x


def array_to_img(x, data_format=None, scale=True):
    """Converts a 3D Numpy array to a PIL Image instance.
    # Arguments
        x: Input Numpy array.
        data_format: Image data format.
        scale: Whether to rescale image values
            to be within [0, 255].
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
        ValueError: if invalid `x` or `data_format` is passed.
    """
    if pil_image is None:
        raise ImportError('Could not import PIL.Image. '
                          'The use of `array_to_img` requires PIL.')
    x = np.asarray(x, dtype=K.floatx())
    if x.ndim != 3:
        raise ValueError('Expected image array to have rank 3 (single image). '
                         'Got array with shape:', x.shape)

    if data_format is None:
        data_format = K.image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Invalid data_format:', data_format)

    # Original Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but target PIL image has format (width, height, channel)
    if data_format == 'channels_first':
        x = x.transpose(1, 2, 0)
    if scale:
        x = x + max(-np.min(x), 0)
        x_max = np.max(x)
        if x_max != 0:
            x /= x_max
        x *= 255
    if x.shape[2] == 3:
        # RGB
        return pil_image.fromarray(x.astype('uint8'), 'RGB')
    elif x.shape[2] == 1:
        # grayscale
        return pil_image.fromarray(x[:, :, 0].astype('uint8'), 'L')
    else:
        raise ValueError('Unsupported channel number: ', x.shape[2])


def img_to_array(img, data_format=None):
    """Converts a PIL Image instance to a Numpy array.
    # Arguments
        img: PIL Image instance.
        data_format: Image data format.
    # Returns
        A 3D Numpy array.
    # Raises
        ValueError: if invalid `img` or `data_format` is passed.
    """
    if data_format is None:
        data_format = K.image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format: ', data_format)
    # Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but original PIL image has format (width, height, channel)
    x = np.asarray(img, dtype=K.floatx())
    if len(x.shape) == 3:
        if data_format == 'channels_first':
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if data_format == 'channels_first':
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise ValueError('Unsupported image shape: ', x.shape)
    return x


def load_img(path, grayscale=False, target_size=None,
             interpolation='nearest'):
    """Loads an image into PIL format.
    # Arguments
        path: Path to image file
        grayscale: Boolean, whether to load the image as grayscale.
        target_size: Either `None` (default to original size)
            or tuple of ints `(img_height, img_width)`.
        interpolation: Interpolation method used to resample the image if the
            target size is different from that of the loaded image.
            Supported methods are "nearest", "bilinear", and "bicubic".
            If PIL version 1.1.3 or newer is installed, "lanczos" is also
            supported. If PIL version 3.4.0 or newer is installed, "box" and
            "hamming" are also supported. By default, "nearest" is used.
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
        ValueError: if interpolation method is not supported.
    """
    if pil_image is None:
        raise ImportError('Could not import PIL.Image. '
                          'The use of `array_to_img` requires PIL.')
    img = pil_image.open(path)
    if grayscale:
        if img.mode != 'L':
            img = img.convert('L')
    else:
        if img.mode != 'RGB':
            img = img.convert('RGB')
    if target_size is not None:
        width_height_tuple = (target_size[1], target_size[0])
        if img.size != width_height_tuple:
            if interpolation not in _PIL_INTERPOLATION_METHODS:
                raise ValueError(
                    'Invalid interpolation method {} specified. Supported '
                    'methods are {}'.format(
                        interpolation,
                        ", ".join(_PIL_INTERPOLATION_METHODS.keys())))
            resample = _PIL_INTERPOLATION_METHODS[interpolation]
            img = img.resize(width_height_tuple, resample)
    return img


def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f)]


class MultiLabelImageDataGenerator(object):
    """Generate minibatches of image data with real-time data augmentation.
    # Arguments
        featurewise_center: set input mean to 0 over the dataset.
        samplewise_center: set each sample mean to 0.
        featurewise_std_normalization: divide inputs by std of the dataset.
        samplewise_std_normalization: divide each input by its std.
        zca_whitening: apply ZCA whitening.
        zca_epsilon: epsilon for ZCA whitening. Default is 1e-6.
        rotation_range: degrees (0 to 180).
        width_shift_range: fraction of total width, if < 1, or pixels if >= 1.
        height_shift_range: fraction of total height, if < 1, or pixels if >= 1.
        brightness_range: the range of brightness to apply
        shear_range: shear intensity (shear angle in degrees).
        zoom_range: amount of zoom. if scalar z, zoom will be randomly picked
            in the range [1-z, 1+z]. A sequence of two can be passed instead
            to select this range.
        channel_shift_range: shift range for each channel.
        fill_mode: points outside the boundaries are filled according to the
            given mode ('constant', 'nearest', 'reflect' or 'wrap'). Default
            is 'nearest'.
            Points outside the boundaries of the input are filled according to the given mode:
                'constant': kkkkkkkk|abcd|kkkkkkkk (cval=k)
                'nearest':  aaaaaaaa|abcd|dddddddd
                'reflect':  abcddcba|abcd|dcbaabcd
                'wrap':  abcdabcd|abcd|abcdabcd
        cval: value used for points outside the boundaries when fill_mode is
            'constant'. Default is 0.
        horizontal_flip: whether to randomly flip images horizontally.
        vertical_flip: whether to randomly flip images vertically.
        rescale: rescaling factor. If None or 0, no rescaling is applied,
            otherwise we multiply the data by the value provided. This is
            applied after the `preprocessing_function` (if any provided)
            but before any other transformation.
        preprocessing_function: function that will be implied on each input.
            The function will run before any other modification on it.
            The function should take one argument:
            one image (Numpy tensor with rank 3),
            and should output a Numpy tensor with the same shape.
        data_format: 'channels_first' or 'channels_last'. In 'channels_first' mode, the channels dimension
            (the depth) is at index 1, in 'channels_last' mode it is at index 3.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
        validation_split: fraction of images reserved for validation (strictly between 0 and 1).
    """

    def __init__(self,
                 featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 zca_epsilon=1e-6,
                 rotation_range=0.,
                 width_shift_range=0.,
                 height_shift_range=0.,
                 brightness_range=None,
                 shear_range=0.,
                 zoom_range=0.,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=False,
                 vertical_flip=False,
                 rescale=None,
                 preprocessing_function=None,
                 data_format=None,
                 validation_split=0.0,
                 label_num=2,
                 input_num=1,
                 constraint_num=0,
                 extra_constraints=[]):
        if data_format is None:
            data_format = K.image_data_format()
        self.featurewise_center = featurewise_center
        self.samplewise_center = samplewise_center
        self.featurewise_std_normalization = featurewise_std_normalization
        self.samplewise_std_normalization = samplewise_std_normalization
        self.zca_whitening = zca_whitening
        self.zca_epsilon = zca_epsilon
        self.rotation_range = rotation_range
        self.width_shift_range = width_shift_range
        self.height_shift_range = height_shift_range
        self.brightness_range = brightness_range
        self.shear_range = shear_range
        self.zoom_range = zoom_range
        self.channel_shift_range = channel_shift_range
        self.fill_mode = fill_mode
        self.cval = cval
        self.horizontal_flip = horizontal_flip
        self.vertical_flip = vertical_flip
        self.rescale = rescale
        self.preprocessing_function = preprocessing_function
        self.label_num = label_num
        self.input_num = input_num
        self.constraint_num = constraint_num
        self.extra_constraints = extra_constraints

        if data_format not in {'channels_last', 'channels_first'}:
            raise ValueError('`data_format` should be `"channels_last"` (channel after row and '
                             'column) or `"channels_first"` (channel before row and column). '
                             'Received arg: ', data_format)
        self.data_format = data_format
        if data_format == 'channels_first':
            self.channel_axis = 1
            self.row_axis = 2
            self.col_axis = 3
        if data_format == 'channels_last':
            self.channel_axis = 3
            self.row_axis = 1
            self.col_axis = 2
        if validation_split and not 0 < validation_split < 1:
            raise ValueError('`validation_split` must be strictly between 0 and 1. '
                             ' Received arg: ', validation_split)
        self._validation_split = validation_split

        self.mean = None
        self.std = None
        self.principal_components = None

        if np.isscalar(zoom_range):
            self.zoom_range = [1 - zoom_range, 1 + zoom_range]
        elif len(zoom_range) == 2:
            self.zoom_range = [zoom_range[0], zoom_range[1]]
        else:
            raise ValueError('`zoom_range` should be a float or '
                             'a tuple or list of two floats. '
                             'Received arg: ', zoom_range)
        if zca_whitening:
            if not featurewise_center:
                self.featurewise_center = True
                warnings.warn('This ImageDataGenerator specifies '
                              '`zca_whitening`, which overrides '
                              'setting of `featurewise_center`.')
            if featurewise_std_normalization:
                self.featurewise_std_normalization = False
                warnings.warn('This ImageDataGenerator specifies '
                              '`zca_whitening` '
                              'which overrides setting of'
                              '`featurewise_std_normalization`.')
        if featurewise_std_normalization:
            if not featurewise_center:
                self.featurewise_center = True
                warnings.warn('This ImageDataGenerator specifies '
                              '`featurewise_std_normalization`, '
                              'which overrides setting of '
                              '`featurewise_center`.')
        if samplewise_std_normalization:
            if not samplewise_center:
                self.samplewise_center = True
                warnings.warn('This ImageDataGenerator specifies '
                              '`samplewise_std_normalization`, '
                              'which overrides setting of '
                              '`samplewise_center`.')

    def flow(self, x, y=None, batch_size=32, shuffle=True, seed=None,
             save_to_dir=None, save_prefix='', save_format='png', subset=None):
        return NumpyArrayIterator(
            x, y, self,
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            data_format=self.data_format,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format,
            subset=subset)

    def flow_from_directory(self, directory,
                            target_size=(256, 256), color_mode='rgb',
                            classes=None, class_mode='categorical',
                            batch_size=32, shuffle=True, seed=None,
                            save_to_dir=None,
                            save_prefix='',
                            save_format='png',
                            follow_links=False,
                            subset=None,
                            interpolation='nearest'):
        return DirectoryIterator(
            directory, self,
            target_size=target_size, color_mode=color_mode,
            classes=classes, class_mode=class_mode,
            data_format=self.data_format,
            batch_size=batch_size, shuffle=shuffle, seed=seed,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format,
            follow_links=follow_links,
            subset=subset,
            interpolation=interpolation)

    def standardize(self, x):
        """Apply the normalization configuration to a batch of inputs.
        # Arguments
            x: batch of inputs to be normalized.
        # Returns
            The inputs, normalized.
        """
        if self.rescale:
            x *= self.rescale
        if self.samplewise_center:
            x -= np.mean(x, keepdims=True)
        if self.samplewise_std_normalization:
            x /= (np.std(x, keepdims=True) + K.epsilon())

        if self.featurewise_center:
            if self.mean is not None:
                x -= self.mean
            else:
                warnings.warn('This ImageDataGenerator specifies '
                              '`featurewise_center`, but it hasn\'t '
                              'been fit on any training data. Fit it '
                              'first by calling `.fit(numpy_data)`.')
        if self.featurewise_std_normalization:
            if self.std is not None:
                x /= (self.std + K.epsilon())
            else:
                warnings.warn('This ImageDataGenerator specifies '
                              '`featurewise_std_normalization`, but it hasn\'t '
                              'been fit on any training data. Fit it '
                              'first by calling `.fit(numpy_data)`.')
        if self.zca_whitening:
            if self.principal_components is not None:
                flatx = np.reshape(x, (-1, np.prod(x.shape[-3:])))
                whitex = np.dot(flatx, self.principal_components)
                x = np.reshape(whitex, x.shape)
            else:
                warnings.warn('This ImageDataGenerator specifies '
                              '`zca_whitening`, but it hasn\'t '
                              'been fit on any training data. Fit it '
                              'first by calling `.fit(numpy_data)`.')
        return x

    def random_transform(self, x, seed=None):
        """Randomly augment a single image tensor.
        # Arguments
            x: 3D tensor, single image.
            seed: random seed.
        # Returns
            A randomly transformed version of the input (same shape).
        """
        # x is a single image, so it doesn't have image number at index 0
        img_row_axis = self.row_axis - 1
        img_col_axis = self.col_axis - 1
        img_channel_axis = self.channel_axis - 1

        if seed is not None:
            np.random.seed(seed)

        # use composition of homographies
        # to generate final transform that needs to be applied
        if self.rotation_range:
            theta = np.deg2rad(np.random.uniform(-self.rotation_range, self.rotation_range))
        else:
            theta = 0

        if self.height_shift_range:
            tx = np.random.uniform(-self.height_shift_range, self.height_shift_range)
            if self.height_shift_range < 1:
                tx *= x.shape[img_row_axis]
        else:
            tx = 0

        if self.width_shift_range:
            ty = np.random.uniform(-self.width_shift_range, self.width_shift_range)
            if self.width_shift_range < 1:
                ty *= x.shape[img_col_axis]
        else:
            ty = 0

        if self.shear_range:
            shear = np.deg2rad(np.random.uniform(-self.shear_range, self.shear_range))
        else:
            shear = 0

        if self.zoom_range[0] == 1 and self.zoom_range[1] == 1:
            zx, zy = 1, 1
        else:
            zx, zy = np.random.uniform(self.zoom_range[0], self.zoom_range[1], 2)

        transform_matrix = None
        if theta != 0:
            rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
            transform_matrix = rotation_matrix

        if tx != 0 or ty != 0:
            shift_matrix = np.array([[1, 0, tx],
                                     [0, 1, ty],
                                     [0, 0, 1]])
            transform_matrix = shift_matrix if transform_matrix is None else np.dot(transform_matrix, shift_matrix)

        if shear != 0:
            shear_matrix = np.array([[1, -np.sin(shear), 0],
                                     [0, np.cos(shear), 0],
                                     [0, 0, 1]])
            transform_matrix = shear_matrix if transform_matrix is None else np.dot(transform_matrix, shear_matrix)

        if zx != 1 or zy != 1:
            zoom_matrix = np.array([[zx, 0, 0],
                                    [0, zy, 0],
                                    [0, 0, 1]])
            transform_matrix = zoom_matrix if transform_matrix is None else np.dot(transform_matrix, zoom_matrix)

        if transform_matrix is not None:
            h, w = x.shape[img_row_axis], x.shape[img_col_axis]
            transform_matrix = transform_matrix_offset_center(transform_matrix, h, w)
            x = apply_transform(x, transform_matrix, img_channel_axis,
                                fill_mode=self.fill_mode, cval=self.cval)

        if self.channel_shift_range != 0:
            x = random_channel_shift(x,
                                     self.channel_shift_range,
                                     img_channel_axis)
        if self.horizontal_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_col_axis)

        if self.vertical_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_row_axis)

        if self.brightness_range is not None:
            x = random_brightness(x, self.brightness_range)

        return x

    def fit(self, x,
            augment=False,
            rounds=1,
            seed=None):
        """Fits internal statistics to some sample data.
        Required for featurewise_center, featurewise_std_normalization
        and zca_whitening.
        # Arguments
            x: Numpy array, the data to fit on. Should have rank 4.
                In case of grayscale data,
                the channels axis should have value 1, and in case
                of RGB data, it should have value 3.
            augment: Whether to fit on randomly augmented samples
            rounds: If `augment`,
                how many augmentation passes to do over the data
            seed: random seed.
        # Raises
            ValueError: in case of invalid input `x`.
        """
        x = np.asarray(x, dtype=K.floatx())
        if x.ndim != 4:
            raise ValueError('Input to `.fit()` should have rank 4. '
                             'Got array with shape: ' + str(x.shape))
        if x.shape[self.channel_axis] not in {1, 3, 4}:
            warnings.warn(
                'Expected input to be images (as Numpy array) '
                'following the data format convention "' + self.data_format + '" '
                                                                              '(channels on axis ' + str(
                    self.channel_axis) + '), i.e. expected '
                                         'either 1, 3 or 4 channels on axis ' + str(self.channel_axis) + '. '
                                                                                                         'However, it was passed an array with shape ' + str(
                    x.shape) +
                ' (' + str(x.shape[self.channel_axis]) + ' channels).')

        if seed is not None:
            np.random.seed(seed)

        x = np.copy(x)
        if augment:
            ax = np.zeros(tuple([rounds * x.shape[0]] + list(x.shape)[1:]), dtype=K.floatx())
            for r in range(rounds):
                for i in range(x.shape[0]):
                    ax[i + r * x.shape[0]] = self.random_transform(x[i])
            x = ax

        if self.featurewise_center:
            self.mean = np.mean(x, axis=(0, self.row_axis, self.col_axis))
            broadcast_shape = [1, 1, 1]
            broadcast_shape[self.channel_axis - 1] = x.shape[self.channel_axis]
            self.mean = np.reshape(self.mean, broadcast_shape)
            x -= self.mean

        if self.featurewise_std_normalization:
            self.std = np.std(x, axis=(0, self.row_axis, self.col_axis))
            broadcast_shape = [1, 1, 1]
            broadcast_shape[self.channel_axis - 1] = x.shape[self.channel_axis]
            self.std = np.reshape(self.std, broadcast_shape)
            x /= (self.std + K.epsilon())

        if self.zca_whitening:
            flat_x = np.reshape(x, (x.shape[0], x.shape[1] * x.shape[2] * x.shape[3]))
            sigma = np.dot(flat_x.T, flat_x) / flat_x.shape[0]
            u, s, _ = linalg.svd(sigma)
            s_inv = 1. / np.sqrt(s[np.newaxis] + self.zca_epsilon)
            self.principal_components = (u * s_inv).dot(u.T)


class Iterator(Sequence):
    """Base class for image data iterators.
    Every `Iterator` must implement the `_get_batches_of_transformed_samples`
    method.
    # Arguments
        n: Integer, total number of samples in the dataset to loop over.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seeding for data shuffling.
    """

    def __init__(self, n, batch_size, shuffle, seed):
        self.n = n
        self.batch_size = batch_size
        self.seed = seed
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.lock = threading.Lock()
        self.index_array = None
        self.index_generator = self._flow_index()

    def _set_index_array(self):
        self.index_array = np.arange(self.n)
        if self.shuffle:
            self.index_array = np.random.permutation(self.n)

    def __getitem__(self, idx):
        if idx >= len(self):
            raise ValueError('Asked to retrieve element {idx}, '
                             'but the Sequence '
                             'has length {length}'.format(idx=idx,
                                                          length=len(self)))
        if self.seed is not None:
            np.random.seed(self.seed + self.total_batches_seen)
        self.total_batches_seen += 1
        if self.index_array is None:
            self._set_index_array()
        index_array = self.index_array[self.batch_size * idx:
                                       self.batch_size * (idx + 1)]
        return self._get_batches_of_transformed_samples(index_array)

    def __len__(self):
        return (self.n + self.batch_size - 1) // self.batch_size  # round up

    def on_epoch_end(self):
        self._set_index_array()

    def reset(self):
        self.batch_index = 0

    def _flow_index(self):
        # Ensure self.batch_index is 0.
        self.reset()
        while 1:
            if self.seed is not None:
                np.random.seed(self.seed + self.total_batches_seen)
            if self.batch_index == 0:
                self._set_index_array()

            current_index = (self.batch_index * self.batch_size) % self.n
            if self.n > current_index + self.batch_size:
                self.batch_index += 1
            else:
                self.batch_index = 0
            self.total_batches_seen += 1
            yield self.index_array[current_index:
                                   current_index + self.batch_size]

    def __iter__(self):
        # Needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)

    def _get_batches_of_transformed_samples(self, index_array):
        """Gets a batch of transformed samples.
        # Arguments
            index_array: array of sample indices to include in batch.
        # Returns
            A batch of transformed samples.
        """
        raise NotImplementedError


class NumpyArrayIterator(Iterator):
    """Iterator yielding data from a Numpy array.
    # Arguments
        x: Numpy array of input data.
        y: Numpy array of targets data.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
        subset: Subset of data (`"training"` or `"validation"`) if
            validation_split is set in ImageDataGenerator.
    """

    def __init__(self, x, y, image_data_generator,
                 batch_size=32, shuffle=False, seed=None,
                 data_format=None,
                 save_to_dir=None, save_prefix='', save_format='png',
                 subset=None):
        if y is not None and len(x) != len(y):
            raise ValueError('`x` (images tensor) and `y` (labels) '
                             'should have the same length. '
                             'Found: x.shape = %s, y.shape = %s' %
                             (np.asarray(x).shape, np.asarray(y).shape))
        if subset is not None:
            if subset not in {'training', 'validation'}:
                raise ValueError('Invalid subset name:', subset,
                                 '; expected "training" or "validation".')
            split_idx = int(len(x) * image_data_generator._validation_split)
            if subset == 'validation':
                x = x[:split_idx]
                if y is not None:
                    y = y[:split_idx]
            else:
                x = x[split_idx:]
                if y is not None:
                    y = y[split_idx:]
        if data_format is None:
            data_format = K.image_data_format()
        self.x = np.asarray(x, dtype=K.floatx())
        if self.x.ndim != 4:
            raise ValueError('Input data in `NumpyArrayIterator` '
                             'should have rank 4. You passed an array '
                             'with shape', self.x.shape)
        channels_axis = 3 if data_format == 'channels_last' else 1
        if self.x.shape[channels_axis] not in {1, 3, 4}:
            warnings.warn('NumpyArrayIterator is set to use the '
                          'data format convention "' + data_format + '" '
                                                                     '(channels on axis ' + str(
                channels_axis) + '), i.e. expected '
                                 'either 1, 3 or 4 channels on axis ' + str(channels_axis) + '. '
                                                                                             'However, it was passed an array with shape ' + str(
                self.x.shape) +
                          ' (' + str(self.x.shape[channels_axis]) + ' channels).')
        if y is not None:
            self.y = np.asarray(y)
        else:
            self.y = None
        self.image_data_generator = image_data_generator
        self.data_format = data_format
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        super(NumpyArrayIterator, self).__init__(x.shape[0], batch_size, shuffle, seed)

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros(tuple([len(index_array)] + list(self.x.shape)[1:]),
                           dtype=K.floatx())
        for i, j in enumerate(index_array):
            x = self.x[j]
            if self.image_data_generator.preprocessing_function:
                x = self.image_data_generator.preprocessing_function(x)
            x = self.image_data_generator.random_transform(x.astype(K.floatx()))
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
        if self.save_to_dir:
            for i, j in enumerate(index_array):
                img = array_to_img(batch_x[i], self.data_format, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                  index=j,
                                                                  hash=np.random.randint(1e4),
                                                                  format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
        if self.y is None:
            return batch_x
        batch_y = self.y[index_array]
        return batch_x, batch_y

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        return self._get_batches_of_transformed_samples(index_array)


def _iter_valid_files(directory, white_list_formats, follow_links):
    """Count files with extension in `white_list_formats` contained in directory.
    # Arguments
        directory: absolute path to the directory
            containing files to be counted
        white_list_formats: set of strings containing allowed extensions for
            the files to be counted.
        follow_links: boolean.
    # Yields
        tuple of (root, filename) with extension in `white_list_formats`.
    """

    def _recursive_list(subpath):
        return sorted(os.walk(subpath, followlinks=follow_links), key=lambda x: x[0])

    for root, _, files in _recursive_list(directory):
        for fname in sorted(files):
            for extension in white_list_formats:
                if fname.lower().endswith('.tiff'):
                    warnings.warn('Using \'.tiff\' files with multiple bands will cause distortion. '
                                  'Please verify your output.')
                if fname.lower().endswith('.' + extension):
                    yield root, fname


def _count_valid_files_in_directory(directory, white_list_formats, split, follow_links):
    """Count files with extension in `white_list_formats` contained in directory.
    # Arguments
        directory: absolute path to the directory
            containing files to be counted
        white_list_formats: set of strings containing allowed extensions for
            the files to be counted.
        split: tuple of floats (e.g. `(0.2, 0.6)`) to only take into
            account a certain fraction of files in each directory.
            E.g.: `segment=(0.6, 1.0)` would only account for last 40 percent
            of images in each directory.
        follow_links: boolean.
    # Returns
        the count of files with extension in `white_list_formats` contained in
        the directory.
    """
    num_files = len(list(_iter_valid_files(directory, white_list_formats, follow_links)))
    if split:
        start, stop = int(split[0] * num_files), int(split[1] * num_files)
    else:
        start, stop = 0, num_files
    return stop - start


def _list_valid_filenames_in_directory(directory, white_list_formats, split,
                                       class_indices, follow_links):
    """List paths of files in `subdir` with extensions in `white_list_formats`.
    # Arguments
        directory: absolute path to a directory containing the files to list.
            The directory name is used as class label and must be a key of `class_indices`.
        white_list_formats: set of strings containing allowed extensions for
            the files to be counted.
        split: tuple of floats (e.g. `(0.2, 0.6)`) to only take into
            account a certain fraction of files in each directory.
            E.g.: `segment=(0.6, 1.0)` would only account for last 40 percent
            of images in each directory.
        class_indices: dictionary mapping a class name to its index.
        follow_links: boolean.
    # Returns
        classes: a list of class indices
        filenames: the path of valid files in `directory`, relative from
            `directory`'s parent (e.g., if `directory` is "dataset/class1",
            the filenames will be ["class1/file1.jpg", "class1/file2.jpg", ...]).
    """
    dirname = os.path.basename(directory)
    if split:
        num_files = len(list(_iter_valid_files(directory, white_list_formats, follow_links)))
        start, stop = int(split[0] * num_files), int(split[1] * num_files)
        valid_files = list(_iter_valid_files(directory, white_list_formats, follow_links))[start: stop]
    else:
        valid_files = _iter_valid_files(directory, white_list_formats, follow_links)

    classes = []
    filenames = []
    for root, fname in valid_files:
        classes.append(class_indices[dirname])
        absolute_path = os.path.join(root, fname)
        relative_path = os.path.join(dirname, os.path.relpath(absolute_path, directory))
        filenames.append(relative_path)

    return classes, filenames


class DirectoryIterator(Iterator):
    """Iterator capable of reading images from a directory on disk.
    # Arguments
        directory: Path to the directory to read images from.
            Each subdirectory in this directory will be
            considered to contain images from one class,
            or alternatively you could specify class subdirectories
            via the `classes` argument.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        target_size: tuple of integers, dimensions to resize input images to.
        color_mode: One of `"rgb"`, `"grayscale"`. Color mode to read images.
        classes: Optional list of strings, names of subdirectories
            containing images from each class (e.g. `["dogs", "cats"]`).
            It will be computed automatically if not set.
        class_mode: Mode for yielding the targets:
            `"binary"`: binary targets (if there are only two classes),
            `"categorical"`: categorical targets,
            `"sparse"`: integer targets,
            `"input"`: targets are images identical to input images (mainly
                used to work with autoencoders),
            `None`: no targets get yielded (only input images are yielded).
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
        subset: Subset of data (`"training"` or `"validation"`) if
            validation_split is set in ImageDataGenerator.
        interpolation: Interpolation method used to resample the image if the
            target size is different from that of the loaded image.
            Supported methods are "nearest", "bilinear", and "bicubic".
            If PIL version 1.1.3 or newer is installed, "lanczos" is also
            supported. If PIL version 3.4.0 or newer is installed, "box" and
            "hamming" are also supported. By default, "nearest" is used.
    """

    def __init__(self, directory, image_data_generator,
                 target_size=(256, 256), color_mode='rgb',
                 classes=None, class_mode='categorical',
                 batch_size=32, shuffle=True, seed=None,
                 data_format=None,
                 save_to_dir=None, save_prefix='', save_format='png',
                 follow_links=False,
                 subset=None,
                 interpolation='nearest'):
        if data_format is None:
            data_format = K.image_data_format()
        self.directory = directory
        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)
        if color_mode not in {'rgb', 'grayscale'}:
            raise ValueError('Invalid color mode:', color_mode,
                             '; expected "rgb" or "grayscale".')
        self.color_mode = color_mode
        self.data_format = data_format
        if self.color_mode == 'rgb':
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (3,)
            else:
                self.image_shape = (3,) + self.target_size
        else:
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (1,)
            else:
                self.image_shape = (1,) + self.target_size
        self.classes = classes
        if class_mode not in {'categorical', 'binary', 'sparse',
                              'input', None}:
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", "input"'
                             ' or None.')
        self.class_mode = class_mode
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        self.interpolation = interpolation

        if subset is not None:
            validation_split = self.image_data_generator._validation_split
            if subset == 'validation':
                split = (0, validation_split)
            elif subset == 'training':
                split = (validation_split, 1)
            else:
                raise ValueError('Invalid subset name: ', subset,
                                 '; expected "training" or "validation"')
        else:
            split = None
        self.subset = subset

        white_list_formats = {'png', 'jpg', 'jpeg', 'bmp', 'ppm', 'tif', 'tiff'}

        # first, count the number of samples and classes
        self.samples = 0

        if not classes:
            classes = []
            for subdir in sorted(os.listdir(directory)):
                if os.path.isdir(os.path.join(directory, subdir)):
                    classes.append(subdir)
        self.num_classes = len(classes)
        self.class_indices = dict(zip(classes, range(len(classes))))

        pool = multiprocessing.pool.ThreadPool()
        function_partial = partial(_count_valid_files_in_directory,
                                   white_list_formats=white_list_formats,
                                   follow_links=follow_links,
                                   split=split)
        self.samples = sum(pool.map(function_partial,
                                    (os.path.join(directory, subdir)
                                     for subdir in classes)))

        print('Found %d images belonging to %d classes.' % (self.samples, self.num_classes))

        # second, build an index of the images in the different class subfolders
        results = []

        self.filenames = []
        self.classes = np.zeros((self.samples,), dtype='int32')
        i = 0
        for dirpath in (os.path.join(directory, subdir) for subdir in classes):
            results.append(pool.apply_async(_list_valid_filenames_in_directory,
                                            (dirpath, white_list_formats, split,
                                             self.class_indices, follow_links)))
        for res in results:
            classes, filenames = res.get()
            self.classes[i:i + len(classes)] = classes
            self.filenames += filenames
            i += len(classes)

        pool.close()
        pool.join()
        super(DirectoryIterator, self).__init__(self.samples, batch_size, shuffle, seed)

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = load_img(os.path.join(self.directory, fname),
                           grayscale=grayscale,
                           target_size=None,
                           interpolation=self.interpolation)
            if self.image_data_generator.preprocessing_function:
                img = self.image_data_generator.preprocessing_function(img)
            if self.target_size is not None:
                width_height_tuple = (self.target_size[1], self.target_size[0])
                if img.size != width_height_tuple:
                    if self.interpolation not in _PIL_INTERPOLATION_METHODS:
                        raise ValueError(
                            'Invalid interpolation method {} specified. Supported '
                            'methods are {}'.format(
                                self.interpolation,
                                ", ".join(_PIL_INTERPOLATION_METHODS.keys())))
                    resample = _PIL_INTERPOLATION_METHODS[self.interpolation]
                    img = img.resize(width_height_tuple, resample)
            x = img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
        # optionally save augmented images to disk for debugging purposes
        if self.save_to_dir:
            for i, j in enumerate(index_array):
                img = array_to_img(batch_x[i], self.data_format, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                  index=j,
                                                                  hash=np.random.randint(1e7),
                                                                  format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
        # build batch of labels
        if self.class_mode == 'input':
            batch_y = batch_x.copy()
        elif self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), self.num_classes), dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x
        batch_x = [batch_x] * self.image_data_generator.input_num
        label_num = self.image_data_generator.label_num
        constraint_num = self.image_data_generator.constraint_num
        extra_constraints = self.image_data_generator.extra_constraints

        return_value = list((batch_x, [batch_y] * label_num))
        if constraint_num > 0:
            return_value[1] += [np.zeros(len(index_array))] * constraint_num
        if len(extra_constraints) > 0:
            # return_value[1] += [(np.expand_dims(batch_y, axis=-1)) for x in extra_constraints]
            for x in extra_constraints:
                v = batch_y
                for i in range(1, x):
                    v = np.expand_dims(v, axis=-1)
                return_value[1] += [v]
        return_value = tuple(return_value)
        return return_value

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        return self._get_batches_of_transformed_samples(index_array)

In [ ]:
#### MODEL ARCHITECTURE BASED ON THE PAPER https://www.sciencedirect.com/science/article/abs/pii/S0893608019303314?fbclid=IwAR0BWRLz3Uy6vma67aJ9hoLXz8Gk_YeOQTsE9t6gJHEaMnl980JCy5FD9sE ###

import keras.initializers as KI
import keras.layers as KL
import keras.losses as KLoss
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import Convolution2D, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.utils import conv_utils

def normalize_data_format(value):
    if value is None:
        value = K.image_data_format()
    data_format = value.lower()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('The `data_format` argument must be one of '
                         '"channels_first", "channels_last". Received: ' +
                         str(value))
    return data_format


class GlobalAttentionPooling2D(Layer):
    def __init__(self, data_format=None, **kwargs):
        super(GlobalAttentionPooling2D, self).__init__(**kwargs)
        self.data_format = normalize_data_format(data_format)

    def compute_output_shape(self, input_shape):
        input_shape = input_shape[0]
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def call(self, inputs, **kwargs):
        inputs_s = inputs[0]
        inputs_m = inputs[1]

        shape = tuple(inputs_s.get_shape().as_list())

        outputs_s = tf.multiply(inputs_s, inputs_m)
        outputs_m = K.repeat_elements(inputs_m, shape[-1], axis=-1)

        outputs_s = K.sum(outputs_s, axis=[1, 2])
        outputs_m = K.sum(outputs_m, axis=[1, 2])

        outputs = outputs_s / outputs_m

        return outputs

    def get_config(self):
        config = {'data_format': self.data_format}
        base_config = super(GlobalAttentionPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def spatial_mask_generate(S, y_old, height, width, name='', mask_max=2. / 3, mask_min=1. / 3, init_bias=-0.5,
                          s_mask_max=9. / 10, s_mask_min=7. / 10):
    y_old_w = KL.Lambda(lambda t: K.expand_dims(t, axis=1))(y_old)
    y_old_w = KL.Lambda(lambda t: K.expand_dims(t, axis=1))(y_old_w)
    y_old_w = KL.Lambda(lambda t: K.repeat_elements(t, rep=height, axis=1))(y_old_w)
    y_old_w = KL.Lambda(lambda t: K.repeat_elements(t, rep=width, axis=2))(y_old_w)

    S = KL.Lambda(lambda t: t - K.min(t, axis=-1, keepdims=True))(S)

    M = KL.multiply([S, y_old_w])
    M = KL.Lambda(lambda t: K.sum(t, axis=-1))(M)

    S_and = KL.Lambda(lambda t: K.mean(t, axis=-1, keepdims=True))(S)
    S_and = KL.Conv2D(1, (1, 1), activation='tanh',
                      name='scale_transform_s_and_1' + name, kernel_initializer=KI.Constant(value=1),
                      bias_initializer=KI.Constant(value=init_bias))(S_and)
    S_and = KL.Conv2D(1, (1, 1), activation='sigmoid',
                      name='scale_transform_s_and_2' + name, kernel_initializer=KI.Constant(value=10),
                      bias_initializer=KI.Constant(value=3))(S_and)

    M = KL.Lambda(lambda t: K.expand_dims(t, axis=-1))(M)
    M = KL.multiply([M, S_and])

    M = KL.Conv2D(1, (1, 1), name='scale_transform_1' + name, kernel_initializer=KI.Constant(value=1),
                  bias_initializer=KI.Constant(value=init_bias), activation='tanh')(M)
    M = KL.Conv2D(1, (1, 1), name='scale_transform_2' + name, kernel_initializer=KI.Constant(value=10),
                  bias_initializer=KI.Constant(value=3))(M)  # use median and mean?
    M = KL.Lambda(lambda t: K.sigmoid(t), name='mask' + name)(M)

    M_loss = KL.Lambda(lambda t: spatial_mask_loss(t, max_value=mask_max, min_value=mask_min), name='M_loss' + name)(M)
    S_and_loss = KL.Lambda(lambda t: spatial_mask_loss(t, max_value=s_mask_max, min_value=s_mask_min),
                           name='S_and_loss' + name)(S_and)
    return M, M_loss, S_and_loss


def spatial_mask_loss(mask, max_value=4. / 5, min_value=1. / 3):
    length = K.cast(K.shape(mask)[1] * K.shape(mask)[2], dtype='float32')
    sum_value = K.sum(mask, axis=[1, 2])

    low_loss = K.maximum(min_value * length - sum_value, 0)
    high_loss = K.maximum(sum_value - max_value * length, 0)

    final_loss = high_loss + low_loss
    final_loss = final_loss / length

    return final_loss


def rank_transform(t):
    t = K.stack(t)
    t = tf.transpose(t, [1, 0, 2])
    return t


def rank_loss(y_true, y_pred):
    margin = 0.05
    satisfy = 0.7
    y_pred = tf.transpose(y_pred, [1, 0, 2])
    y_true = tf.squeeze(y_true, axis=[2])
    p1 = y_pred[0]
    p2 = y_pred[1]
    p1 = tf.multiply(p1, y_true)
    p2 = tf.multiply(p2, y_true)
    p1m = K.max(p1, axis=-1)
    p2m = K.max(p2, axis=-1)
    rank1 = K.maximum(0.0, p1m - p2m + margin)
    rank2 = K.maximum(0.0, -(p1m + p2m - 2 * satisfy))
    rank = K.minimum(rank1, rank2)
    return rank


def cross_network_similarity_loss(y_true, y_pred):
    y_pred = tf.transpose(y_pred, [1, 0, 2])
    p1 = y_pred[0]
    p2 = y_pred[1]
    kl = KLoss.kullback_leibler_divergence(p1, p2)
    return tf.maximum(0.0, kl - 0.15)


def entropy(pk):
    pk = pk + 0.00001
    e = -tf.reduce_sum(pk * tf.math.log(pk), axis=1)
    return e


def entropy_add(t):
    A1 = t[0]
    A2 = t[1]
    pk1 = t[2]
    pk2 = t[3]
    e1 = entropy(pk1)
    e2 = entropy(pk2)
    mp1 = e2 / (e1 + e2)
    mp2 = 1 - mp1
    mp1 = tf.expand_dims(mp1, axis=1)
    mp2 = tf.expand_dims(mp2, axis=1)
    A1 = A1 * mp1
    A2 = A2 * mp2
    A = (A1 + A2) * 2
    return A


def build_global_attention_pooling_model_cascade_attention(base_network, class_num):
    height, width, depth = base_network[0].output_shape[1:]

    feature_map_step_1 = base_network[0].output

    S = Convolution2D(class_num, (1, 1), name='conv_class')(feature_map_step_1)
    A = GlobalAveragePooling2D()(S)

    y_old = KL.Softmax(name='output_1')(A)
    M, M_loss, S_and_loss = spatial_mask_generate(S, y_old, height, width, mask_max=1. / 2, mask_min=1. / 4)

    feature_map_step_2 = base_network[1].output

    S_new = Convolution2D(class_num, (1, 1), name='conv_class_filtered')(feature_map_step_2)
    A_new = GlobalAttentionPooling2D()([S_new, M])

    y_new = KL.Softmax(name='output_2')(A_new)

    r_loss = KL.Lambda(lambda t: rank_transform(t), name='Rank_loss')([y_old, y_new])
    cns_loss = KL.Lambda(lambda t: t, name='Cross_network_similarity_loss')(r_loss)

    x = KL.concatenate([feature_map_step_1, feature_map_step_2])
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024, activation='relu')(x)
    x = Dense(class_num)(x)
    y_all = KL.Softmax(name='output_3')(x)

    A_final = KL.Lambda(lambda t: entropy_add(t))([x, A_new, y_all, y_new])
    # output_5 is the final output
    y = KL.Softmax(name='output_5')(A_final)

    r2_loss = KL.Lambda(lambda t: rank_transform(t), name='Rank_2_loss')([y_old, y])
    r3_loss = KL.Lambda(lambda t: rank_transform(t), name='Rank_3_loss')([y_new, y])

    for layer in base_network[1].layers:
        layer._name += '_2'

    model = Model(inputs=[base_network[0].input, base_network[1].input],
                  outputs=[y_old, y_new, y_all, y, M_loss, S_and_loss,
                           r_loss, cns_loss,
                           r2_loss, r3_loss
                           ])

    model.summary()
    return model

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings

import keras.callbacks as KC
import numpy as np

try:
    import requests
except ImportError:
    requests = None


class ModelCheckpoint(KC.Callback):
    """Save the model after every epoch.
    `filepath` can contain named formatting options,
    which will be filled the value of `epoch` and
    keys in `logs` (passed in `on_epoch_end`).
    For example: if `filepath` is `weights.{epoch:02d}-{val_loss:.2f}.hdf5`,
    then the model checkpoints will be saved with the epoch number and
    the validation loss in the filename.
    # Arguments
        filepath: string, path to save the model file.
        monitor: quantity to monitor.
        verbose: verbosity mode, 0 or 1.
        save_best_only: if `save_best_only=True`,
            the latest best model according to
            the quantity monitored will not be overwritten.
        mode: one of {auto, min, max}.
            If `save_best_only=True`, the decision
            to overwrite the current save file is made
            based on either the maximization or the
            minimization of the monitored quantity. For `val_acc`,
            this should be `max`, for `val_loss` this should
            be `min`, etc. In `auto` mode, the direction is
            automatically inferred from the name of the monitored quantity.
        save_weights_only: if True, then only the model's weights will be
            saved (`model.save_weights(filepath)`), else the full model
            is saved (`model.save(filepath)`).
        period: Interval (number of epochs) between checkpoints.
    """

    def __init__(self, filepath, single_model, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        super(ModelCheckpoint, self).__init__()
        self.monitor = monitor
        self.verbose = verbose
        self.filepath = filepath
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.period = period
        self.epochs_since_last_save = 0
        self.single_model = single_model

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s available, '
                                  'skipping.' % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch + 1, self.monitor, self.best,
                                     current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.single_model.save_weights(filepath, overwrite=True)
                        else:
                            self.single_model.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s did not improve' %
                                  (epoch + 1, self.monitor))
            else:
                if self.verbose > 0:
                    print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.single_model.save_weights(filepath, overwrite=True)
                else:
                    self.single_model.save(filepath, overwrite=True)

# Steps

In [ ]:
import numpy as np
import os
from keras import applications
from keras import optimizers
import tensorflow as tf
import math
import keras.backend as KB
from keras_preprocessing.image import ImageDataGenerator
#from keras.callbacks import ModelCheckpoint, EarlyStopping
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value) # tensorflow 2.x
# tf.set_random_seed(seed_value) # tensorflow 1.x

# Split Data To make Bagging 

In [ ]:
labels = {}
path = '../input/bird-dasat/bird_dataset/train_images/'
for label in os.listdir(path):
    labels[label + "/"] = [x for x in os.listdir(path + label)]

In [ ]:
folders = ['./split_3', './split_1', './split_2']
os.mkdir('./split_2')
os.mkdir('./split_1')
os.mkdir('./split_3')

In [ ]:
import shutil
import random
for label in labels.keys():
    filenames = labels[label]
    splits = [(0,38),(20, len(filenames)), (10, len(filenames)-10)]
    random.shuffle(filenames)
    for i,fold in enumerate(folders):
        os.mkdir(fold + '/' + label)
        for file in filenames[splits[i][0]:splits[i][1]]:
            shutil.copyfile(path + label + file, fold + '/' + label + file)    

# Data

## TRAIN SPLIT 1

In [ ]:
img_rows, img_cols = (334, 334)
batch_size = 16
batch_size = 16

# prepare data augmentation configuration
train_datagen = MultiLabelImageDataGenerator(
       rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

test_datagen = MultiLabelImageDataGenerator(rescale=1./255,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

train_generator = train_datagen.flow_from_directory(
        './split_1',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        interpolation='bilinear',
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        shuffle=False,
        interpolation='bilinear',
        class_mode='categorical')

In [ ]:
class_num = 20
img_width, img_height = 334, 334

modelvgg = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
modelvgg2 = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#for layer in modelvgg.layers[:len(modelvgg.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg.layers[len(modelvgg.layers)-10:]:
    #layer.trainable = True
#for layer in modelvgg2.layers[:len(modelvgg2.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg2.layers[len(modelvgg2.layers)-10:]:
    #layer.trainable = True

#modelvgg.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
#modelvgg2.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')  # As it's forbidden to use Inaturalist pretrained model
print('Model loaded.')

model = build_global_attention_pooling_model_cascade_attention([modelvgg, modelvgg2], class_num)


model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy',
    'categorical_crossentropy',
    'mean_squared_error', 'mean_squared_error',
    rank_loss,
    cross_network_similarity_loss,
    rank_loss,
    rank_loss,
    ],
# optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=False),
optimizer=optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=1e-08, decay=0.000001),
metrics={'output_1': ['accuracy', 'top_k_categorical_accuracy'],
       'output_2': ['accuracy', 'top_k_categorical_accuracy'],
       'output_3': ['accuracy', 'top_k_categorical_accuracy'],
       # 'output_4': ['accuracy', 'top_k_categorical_accuracy'],
       'output_5': ['accuracy', 'top_k_categorical_accuracy'],
       })

In [ ]:
checkpoint = ModelCheckpoint("./attention_cascade_split1.h5", model, monitor='val_output_5_accuracy',
                                              verbose=1, save_best_only=True, mode='max')
nb_train_samples = 682
nb_validation_samples= 103
epochs=15
batch_size=16

history = model.fit(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.load_weights('./attention_cascade_split1.h5')
model.evaluate(validation_generator)

Split 1 : Validation Accuracy = 0.8932

## Split 2

In [ ]:
img_rows, img_cols = (334, 334)
batch_size = 16
batch_size = 16

# prepare data augmentation configuration
train_datagen = MultiLabelImageDataGenerator(
       rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

test_datagen = MultiLabelImageDataGenerator(rescale=1./255,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

train_generator = train_datagen.flow_from_directory(
        './split_2',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        interpolation='bilinear',
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        shuffle=False,
        interpolation='bilinear',
        class_mode='categorical')

In [ ]:
class_num = 20
img_width, img_height = 334, 334

modelvgg = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
modelvgg2 = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#for layer in modelvgg.layers[:len(modelvgg.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg.layers[len(modelvgg.layers)-10:]:
    #layer.trainable = True
#for layer in modelvgg2.layers[:len(modelvgg2.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg2.layers[len(modelvgg2.layers)-10:]:
    #layer.trainable = True

#modelvgg.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
#modelvgg2.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')  # As it's forbidden to use Inaturalist pretrained model
print('Model loaded.')

model_1 = build_global_attention_pooling_model_cascade_attention([modelvgg, modelvgg2], class_num)


model_1.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy',
    'categorical_crossentropy',
    'mean_squared_error', 'mean_squared_error',
    rank_loss,
    cross_network_similarity_loss,
    rank_loss,
    rank_loss,
    ],
# optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=False),
optimizer=optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=1e-08, decay=0.000001),
metrics={'output_1': ['accuracy', 'top_k_categorical_accuracy'],
       'output_2': ['accuracy', 'top_k_categorical_accuracy'],
       'output_3': ['accuracy', 'top_k_categorical_accuracy'],
       # 'output_4': ['accuracy', 'top_k_categorical_accuracy'],
       'output_5': ['accuracy', 'top_k_categorical_accuracy'],
       })

In [ ]:
checkpoint = ModelCheckpoint("./attention_cascade_split2.h5", model_1, monitor='val_output_5_accuracy',
                                              verbose=1, save_best_only=True, mode='max')
nb_train_samples = 682
nb_validation_samples= 103
epochs=15
batch_size=16

history = model_1.fit(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
model_1.load_weights('./attention_cascade_split2.h5')
model_1.evaluate(validation_generator)

Split 2 accuracy : 0.8835

# Split 3

In [ ]:
img_rows, img_cols = (334, 334)
batch_size = 16
batch_size = 16

# prepare data augmentation configuration
train_datagen = MultiLabelImageDataGenerator(
       rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

test_datagen = MultiLabelImageDataGenerator(rescale=1./255,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

train_generator = train_datagen.flow_from_directory(
        './split_3',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        interpolation='bilinear',
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        shuffle=False,
        interpolation='bilinear',
        class_mode='categorical')

In [ ]:
class_num = 20
img_width, img_height = 334, 334

modelvgg = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
modelvgg2 = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
#for layer in modelvgg.layers[:len(modelvgg.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg.layers[len(modelvgg.layers)-10:]:
    #layer.trainable = True
#for layer in modelvgg2.layers[:len(modelvgg2.layers)-10]:
    #layer.trainable = False
#for layer in modelvgg2.layers[len(modelvgg2.layers)-10:]:
    #layer.trainable = True

#modelvgg.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
#modelvgg2.load_weights('../input/inaturalist/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')  # As it's forbidden to use Inaturalist pretrained model
print('Model loaded.')

model_2 = build_global_attention_pooling_model_cascade_attention([modelvgg, modelvgg2], class_num)


model_2.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy',
    'categorical_crossentropy',
    'mean_squared_error', 'mean_squared_error',
    rank_loss,
    cross_network_similarity_loss,
    rank_loss,
    rank_loss,
    ],
# optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=False),
optimizer=optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=1e-08, decay=0.000001),
metrics={'output_1': ['accuracy', 'top_k_categorical_accuracy'],
       'output_2': ['accuracy', 'top_k_categorical_accuracy'],
       'output_3': ['accuracy', 'top_k_categorical_accuracy'],
       # 'output_4': ['accuracy', 'top_k_categorical_accuracy'],
       'output_5': ['accuracy', 'top_k_categorical_accuracy'],
       })

In [ ]:
checkpoint = ModelCheckpoint("./attention_cascade_split3.h5", model_2, monitor='val_output_5_accuracy',
                                              verbose=1, save_best_only=True, mode='max')
nb_train_samples = 760
nb_validation_samples= 103
epochs=15
batch_size=16

history = model_2.fit(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
model_2.load_weights('./attention_cascade_split3.h5')
model_2.evaluate(validation_generator)

Split 3 : Accuracy : 0.9029

# Predictions

In [ ]:
test_datagen = MultiLabelImageDataGenerator(rescale=1./255,
        input_num=2,
        label_num=4,
        constraint_num=2,
        extra_constraints=[2, 2, 2, 2])

test_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images',
        target_size=(img_rows, img_cols),
        batch_size=1,
        shuffle=False,
        interpolation='bilinear',
        class_mode='categorical')

In [ ]:
predictions_0 = model.predict(test_generator)
predictions_1 = model_1.predict(test_generator)
predictions_2 = model_2.predict(test_generator)
pred_0 = predictions_0[3]
pred_1 = predictions_1[3]
pred_2 = predictions_2[3]

# Averaging

In [ ]:
pred_tot = pred_0 + pred_1 + pred_2
cat = np.argmax(pred_tot, axis=1)

In [ ]:
import pandas as pd
id_ = [x.split('/')[1][:-4] for x in test_generator.filenames]
df = pd.DataFrame({'Id' : id_, 'Category': cat})
df.to_csv('./submissions_9.csv', index=False)

0.69 ...

## Voting

In [ ]:
cat_ = []
def most_frequent(List): 
    return max(set(List), key = List.count)
for X in zip(pred_0, pred_1, pred_2):
    labels = [np.argmax(t) for t in X]
    probs = [np.max(t) for t in X]
    if len(list(set(labels))) < len(labels):
        cat_.append(most_frequent(labels))
    else:
        max_indice = probs.index(max(probs))
        cat_.append(labels[max_indice])
        

In [ ]:
df = pd.DataFrame({'Id' : id_, 'Category': cat_})
df.to_csv('./submissions_10.csv', index=False)

0.70967